## Instructions
Opening the notebook with Colab is strongly recommmended. Feel free to always select `Run all` since the pipeline has taken care of installing/importing packages as well as dependencies.    

We frist showcase the use of `dice_ml` library to apply DiCE with the adult dataset using pretrained PyTorch model as the classifier, and demonstrate customized queries to have DiCE generate a set of recourses for a negatively impacted user to choose from. 

Then we evaluate three algorithmic recourse methods using the library `relax` on three datasets after training simple neural network models. When running this notebook, please change runtime type to GPU to speed the training and generating process.

In [1]:
import importlib.util
if importlib.util.find_spec('relax') is None:
  !pip install jax-relax
if importlib.util.find_spec('dice_ml') is None:
  !pip install dice_ml

import jax
import dice_ml
from sklearn.model_selection import train_test_split
from dice_ml.utils import helpers  # helper functions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.1 MB/s eta 0:00:00
  Created wheel for test-tube: filename=test_tube-0.7.5-py3-none-any.whl size=25356 sha256=cfbee0882eddfa16254c2fb55180b17b23def99195dd0968bd4a9181bdaee419
  Stored in directory: /root/.cache/pip/wheels/28/d4/8b/1aeb47c

In [2]:
dataset = helpers.load_adult_income_dataset()
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [3]:
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

Split the train and test dataset and specify the pretrained PyTorch model to be used.

In [4]:
target = dataset["income"]
Xtrain, Xtest, y_train, y_test = train_test_split(dataset, target, test_size=0.2, random_state=0,stratify=target)
x_train = Xtrain.drop('income', axis=1)
x_test = Xtest.drop('income', axis=1)

In [5]:
data_config = dice_ml.Data(dataframe=Xtrain, continuous_features=['age', 'hours_per_week'], outcome_name='income')

pytorch_pretrained = helpers.get_adult_income_modelpath(backend='PYT')
model_config = dice_ml.Model(model_path=pytorch_pretrained, backend='PYT',  func="ohe-min-max")

dice_cf = dice_ml.Dice(data_config, model_config, method="gradient")


Generate recourses and show corresponding changes to the features.

In [6]:
dice_res = dice_cf.generate_counterfactuals(x_test[0:1], total_CFs=5, desired_class="opposite")

dice_res.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:18<00:00, 18.55s/it]

Diverse Counterfactuals found! total time taken: 00 min 17 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29.0,Private,HS-grad,Married,Blue-Collar,White,Female,38.0,0.021



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,Self-Employed,Doctorate,-,White-Collar,-,-,-,1
1,78.0,-,Prof-school,-,-,-,Male,-,1
2,41.0,-,Prof-school,-,Sales,-,Male,-,1
3,45.0,-,Masters,-,Other/Unknown,-,-,-,1
4,50.0,-,Doctorate,-,-,-,-,69.0,1


Now we can specify features that we allow to change, with other feature staying as they were. 

In [7]:
dice_res_specified = dice_cf.generate_counterfactuals(x_test[0:1], total_CFs=5, desired_class="opposite", features_to_vary=["education", "hours_per_week"])

dice_res_specified.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:20<00:00, 20.62s/it]

Diverse Counterfactuals found! total time taken: 00 min 20 sec
Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29.0,Private,HS-grad,Married,Blue-Collar,White,Female,38.0,0.021



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Prof-school,-,-,-,-,32.0,1
1,-,-,Doctorate,-,-,-,-,43.0,1
2,-,-,Masters,-,-,-,-,61.0,1
3,-,-,Prof-school,-,-,-,-,81.0,1
4,-,-,Doctorate,-,-,-,-,99.0,1


## Procedure
We will follow a general procedure as:

1. Specify the dataset to run the algorithmic recourse on. Datasets: *adult*, *oulad*, *heloc*. 

2. Train a simple neural network model with 3 hidden layers as the classifier (one for each dataset). 

3. Set configurations for three algorithmic recourse methods and use them to generate counterfactuals

4. Evaluate the algorithmic recourse methods based on metrics as validity and proximity. 

In [8]:
from relax.data import load_data
from relax.module import PredictiveTrainingModuleConfigs, PredictiveTrainingModule
from relax.trainer import TrainingConfigs, train_model

### Adult
Load the *adult* dataset, set up the model configuration and training configuration and train a neural network model with 3 hidden layers. 

In [9]:
adult_dataset = load_data('adult')

model_config_adult = PredictiveTrainingModuleConfigs(
    lr=0.001, # Learning rate
    sizes=[50, 10, 50], # Sizes of hidden layers
    dropout_rate=0.3 # Dropout rate
)


adult_model = PredictiveTrainingModule(model_config_adult)

train_config_adult = TrainingConfigs(
    n_epochs=20, 
    batch_size=256, 
    monitor_metrics='val/val_loss', 
    logger_name='pred' 
)


params_adult, _ = train_model(adult_model, adult_dataset, train_config_adult)

pred_fn_adult = adult_model.pred_fn

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
Epoch 19: 100%|██████████| 96/96 [00:00<00:00, 287.71batch/s, train/train_loss_1=0.0663]


In [10]:
from relax.methods import VanillaCF, DiverseCF, ProtoCF
from relax.evaluate import generate_cf_explanations
from relax.evaluate import benchmark_cfs

Initialize three algorithmic recourse methods and generate recourses for *adult*.

In [11]:
wachter_cf_adult = VanillaCF()
diversecf_cf_adult = DiverseCF()
protocf_cf_adult = ProtoCF()

wachter_adult = generate_cf_explanations(
    wachter_cf_adult, adult_dataset, pred_fn_adult, 
    pred_fn_args={
        'params': params_adult, 'rng_key': jax.random.PRNGKey(0)
    }
)

diverse_adult = generate_cf_explanations(
    diversecf_cf_adult, adult_dataset, pred_fn_adult, 
    pred_fn_args={
        'params': params_adult, 'rng_key': jax.random.PRNGKey(0)
    }
)

proto_adult = generate_cf_explanations(
    protocf_cf_adult, adult_dataset, pred_fn_adult, 
    t_configs=dict(
        n_epochs=5, batch_size=128
    ), 
    pred_fn_args={
        'params': params_adult, 'rng_key': jax.random.PRNGKey(0)
    }
)

100%|██████████| 1000/1000 [00:02<00:00, 399.97it/s]


ProtoCF contains parametric models. Starts training before generating explanations...


/usr/local/lib/python3.10/dist-packages/relax/_ckpt_manager.py:47: UserWarning: `monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.
  warnings.warn(
100%|██████████| 1000/1000 [00:02<00:00, 367.34it/s]


Show results with respect to classifier accuracy, recourse validity and recourse cost.

In [12]:
benchmark_cfs([wachter_adult, diverse_adult, proto_adult])

acc  validity  proximity
adult VanillaCF  0.824346  0.933178   8.139759
      DiverseCF  0.824346  0.508537   2.905395
      ProtoCF    0.824346  0.824592   6.932194

### Oulad
Load the *oulad* dataset, set up the model configuration and training configuration and train a neural network model with 3 hidden layers. 

In [13]:
oulad_dataset = load_data('oulad')

model_config_oulad = PredictiveTrainingModuleConfigs(
    lr=0.001, # Learning rate
    sizes=[50, 10, 50], # Sizes of hidden layers
    dropout_rate=0.3 # Dropout rate
)


oulad_model = PredictiveTrainingModule(model_config_oulad)

train_config_oulad = TrainingConfigs(
    n_epochs=20, 
    batch_size=256, 
    monitor_metrics='val/val_loss',
    logger_name='pred' 
)

params_oulad, _ = train_model(oulad_model, oulad_dataset, train_config_oulad)

pred_fn_oulad = oulad_model.pred_fn

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
Epoch 19: 100%|██████████| 96/96 [00:00<00:00, 281.99batch/s, train/train_loss_1=0.0169]


Initialize three algorithmic recourse methods and generate recourses for *oulad*.

In [14]:
wachter_cf_oulad = VanillaCF()
diversecf_cf_oulad = DiverseCF()
protocf_cf_oulad = ProtoCF()

wachter_oulad = generate_cf_explanations(
    wachter_cf_oulad, oulad_dataset, pred_fn_oulad, 
    pred_fn_args={
        'params': params_oulad, 'rng_key': jax.random.PRNGKey(0)
    }
)

diverse_oulad = generate_cf_explanations(
    diversecf_cf_oulad, oulad_dataset, pred_fn_oulad, 
    pred_fn_args={
        'params': params_oulad, 'rng_key': jax.random.PRNGKey(0)
    }
)

proto_oulad = generate_cf_explanations(
    protocf_cf_oulad, oulad_dataset, pred_fn_oulad, 
    pred_fn_args={
        'params': params_oulad, 'rng_key': jax.random.PRNGKey(0)
    }
)

100%|██████████| 1000/1000 [00:02<00:00, 370.19it/s]


ProtoCF contains parametric models. Starts training before generating explanations...


/usr/local/lib/python3.10/dist-packages/relax/_ckpt_manager.py:47: UserWarning: `monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.
  warnings.warn(
100%|██████████| 1000/1000 [00:02<00:00, 354.96it/s]


Show results with respect to classifier accuracy, recourse validity and recourse cost.

In [15]:
benchmark_cfs([wachter_oulad, diverse_oulad, proto_oulad])

acc  validity  proximity
OULAD VanillaCF  0.928826  0.997423  12.241540
      DiverseCF  0.928826  0.981838   1.067762
      ProtoCF    0.928826  0.774328   4.209029

### Heloc
Load the *heloc* dataset, set up the model configuration and training configuration and train a neural network model with 3 hidden layers. 

In [16]:
heloc_dataset = load_data('heloc')

model_config_heloc = PredictiveTrainingModuleConfigs(
    lr=0.0005, # Learning rate
    sizes=[100, 40, 100], # Sizes of hidden layers
    dropout_rate=0.5 # Dropout rate
)


heloc_model = PredictiveTrainingModule(model_config_heloc)

train_config_heloc = TrainingConfigs(
    n_epochs=40, 
    batch_size=256, 
    monitor_metrics='val/val_loss', 
    logger_name='pred' 
)

params_heloc, _ = train_model(heloc_model, heloc_dataset, train_config_heloc)

pred_fn_heloc = heloc_model.pred_fn

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
Epoch 39: 100%|██████████| 31/31 [00:00<00:00, 266.70batch/s, train/train_loss_1=0.114]


Initialize three algorithmic recourse methods and generate recourses for *heloc*.

In [17]:
wachter_cf_heloc = VanillaCF()
diversecf_cf_heloc = DiverseCF()
protocf_cf_heloc = ProtoCF()

wachter_heloc = generate_cf_explanations(
    wachter_cf_heloc, heloc_dataset, pred_fn_heloc, 
    pred_fn_args={
        'params': params_heloc, 'rng_key': jax.random.PRNGKey(0)
    }
)

diverse_heloc = generate_cf_explanations(
    diversecf_cf_heloc, heloc_dataset, pred_fn_heloc, 
    pred_fn_args={
        'params': params_heloc, 'rng_key': jax.random.PRNGKey(0)
    }
)

proto_heloc = generate_cf_explanations(
    protocf_cf_heloc, heloc_dataset, pred_fn_heloc, 
    pred_fn_args={
        'params': params_heloc, 'rng_key': jax.random.PRNGKey(0)
    }
)

100%|██████████| 1000/1000 [00:02<00:00, 421.32it/s]


ProtoCF contains parametric models. Starts training before generating explanations...


/usr/local/lib/python3.10/dist-packages/relax/_ckpt_manager.py:47: UserWarning: `monitor_metrics` is not specified in `CheckpointManager`. No checkpoints will be stored.
  warnings.warn(
100%|██████████| 1000/1000 [00:02<00:00, 412.05it/s]


Show results with respect to classifier accuracy, recourse validity and recourse cost.

In [18]:
benchmark_cfs([wachter_heloc, diverse_heloc, proto_heloc])

acc  validity  proximity
HELOC VanillaCF  0.711663  0.999618   5.203861
      DiverseCF  0.711663  0.999618   3.885279
      ProtoCF    0.711663  0.896749   1.969762